In [1]:
%load_ext dotenv
%dotenv

In [12]:
import json
import os
import requests
import pandas as pd
from web3 import Web3

In [3]:
w3 = Web3(Web3.HTTPProvider(os.environ['GOERLI_RPC']))
w3.isConnected()

True

In [4]:
CONTRACT_POOL = "0xAf593430b86a0560818a9dF5858B14dDC469Ab98";
CONTRACT_dNFT = "0x0eDb1483320DbdA0C804C1F5673F1733382e37E7";

In [5]:
pool = w3.eth.contract(address=CONTRACT_POOL, abi=json.load(open("abi/Pool.json"))["abi"])
dnft = w3.eth.contract(address=CONTRACT_dNFT, abi=json.load(open("abi/dNFT.json"))["abi"])

In [7]:
INFURA_KEY = os.environ["ETHERSCAN_API"]
url = f"https://api-goerli.etherscan.io/api?module=account&action=txlist&address={CONTRACT_POOL}&startblock=0&endblock=99999999&sort=asc&apikey={INFURA_KEY}"
url

'https://api-goerli.etherscan.io/api?module=account&action=txlist&address=0xAf593430b86a0560818a9dF5858B14dDC469Ab98&startblock=0&endblock=99999999&sort=asc&apikey=TECYJEKT36HQRFI9FA5GESVDQTQGUKJZPX'

In [8]:
r = requests.get(url=url)
txs = r.json()['result']

In [25]:
blockNumbers = []
for tx in txs:
    if tx["functionName"] == "sync()":
        blockNumbers.append(int(tx["blockNumber"]))
# add one block after contract is deployed
blockNumbers.insert(0, 8085008)
# add one block before sync is called
blockNumbers.insert(1, blockNumbers[1]-1)
blockNumbers

[8085008, 8095105, 8095106, 8095157, 8099237, 8099246, 8099251]

In [26]:
nfts = []

for blockNumber in blockNumbers:
    print(f"block: {blockNumber}")
    numberOfMints = dnft.functions.numberOfMints().call(block_identifier=blockNumber);
    for i in range(numberOfMints):
        if i % 10 == 0: print(f"nft: {i} / {numberOfMints}")
        tokenId = dnft.functions.tokenByIndex(i) .call(block_identifier=blockNumber)
        nft     = dnft.functions.idToNft(tokenId).call(block_identifier=blockNumber)

        nfts.append({"tokenId": tokenId, 
                     "blockNumber": blockNumber, 
                     "withdrawn":nft[0],
                     "deposit":nft[1],
                     "xp":nft[2]})
nfts

block: 8085008
nft: 0 / 8
block: 8095105
nft: 0 / 20
nft: 10 / 20
block: 8095106
nft: 0 / 21
nft: 10 / 21
nft: 20 / 21
block: 8095157
nft: 0 / 22
nft: 10 / 22
nft: 20 / 22
block: 8099237
nft: 0 / 22
nft: 10 / 22
nft: 20 / 22
block: 8099246
nft: 0 / 22
nft: 10 / 22
nft: 20 / 22
block: 8099251
nft: 0 / 22
nft: 10 / 22
nft: 20 / 22


[{'tokenId': 0,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900300},
 {'tokenId': 1,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900299},
 {'tokenId': 2,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900298},
 {'tokenId': 3,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900297},
 {'tokenId': 4,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900296},
 {'tokenId': 5,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900295},
 {'tokenId': 6,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900294},
 {'tokenId': 7,
  'blockNumber': 8085008,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900293},
 {'tokenId': 0,
  'blockNumber': 8095105,
  'withdrawn': 0,
  'deposit': 0,
  'xp': 900300},
 {'tokenId': 1,
  'blockNumber': 8095105,
  'withdrawn': 100000000000000000000,
  'deposit': 23374000000000000000,
  'xp': 900299},
 {'tokenId': 2,
  'blockNumber'

In [27]:
df = pd.DataFrame(nfts)
df.to_csv("nfts.csv")